In [13]:
#### load / create a feature dictionary
#### load / create a model space
#### built a routine for unraveling the feature space
#### build theano method for generating model-space tensors from unraveled feature spaces
#### build theano method for generating predictions from unraveled fs
#### build theano method for predicted/measured error across *voxels*, not trials
#### build theano method for gradient w.r.t flattened feature maps
#### refactor model-training machinery to run decoding with voxel resampling and early stopping.

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from theano import tensor as tnsr
from theano import function
from theano import pp
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
T = 10
D = 3
S = 20
V = 101
B = 12

feature_map_tnsr = tnsr.tensor4('feature_map_tnsr') ##T x D x stim_size x stim_size

population_rf_stack = tnsr.tensor3('pop_rf_stack_tnsr') # V x stim_size x stim_size

feature_map_means = tnsr.matrix('feature_map_means') ##D x V
feature_map_stdev = tnsr.matrix('feature_map_stdev') ##D x V

population_model_space_tnsr = (tnsr.log(1+tnsr.sqrt(tnsr.tensordot(feature_map_tnsr,  ##T x D x S x S
                         population_rf_stack,                   ##    V x S x S
					     axes = [[2,3],[1,2]],
					     ))) - feature_map_means) - feature_map_stdev

pmst = function(inputs=[feature_map_tnsr, population_rf_stack, feature_map_means, feature_map_stdev], outputs = population_model_space_tnsr)
##gives T x B x D x V


fmt = np.random.rand(T,D,S,S).astype('float32')
prfs = np.random.rand(V, S, S).astype('float32')
fmm = np.random.rand(D,V).astype('float32')
fms = np.random.rand(D,V).astype('float32')

print pmst(fmt, prfs, fmm, fms).shape
print (T, D, V)

population_feature_weight_tnsr = tnsr.matrix('NU')
population_prediction_tnsr = (population_model_space_tnsr*population_feature_weight_tnsr).sum(axis=1)    
                                            
                                            


ppt = function(inputs = [population_model_space_tnsr, population_feature_weight_tnsr],
              outputs = population_prediction_tnsr)

(10, 3, 101)
(10, 3, 101)


In [16]:
pfwt = np.random.rand(D,V).astype('float32')
X = pmst(fmt,prfs, fmm, fms)

print ppt(X,pfwt).shape
##gives T x V which is correct

(10, 101)


In [17]:
voxel_data_tnsr = tnsr.matrix('voxel_data_tnsr')
population_prediction_error = tnsr.sqr(population_prediction_tnsr - voxel_data_tnsr).sum() 

In [18]:
ppe_func = function(inputs=[population_prediction_tnsr, voxel_data_tnsr],
                    outputs=population_prediction_error)

In [19]:
vdt = np.random.rand(T,V).astype('float32')
ppe_func(ppt(X, pfwt), vdt).shape

()

In [20]:
##grad_population_prediction_error_wrt_feature_map
gfm = tnsr.grad(population_prediction_error,feature_map_tnsr) ##<<the summing trick above makes this easy. 

compute_fm_grad = function(inputs = [voxel_data_tnsr,
                                     feature_map_tnsr, population_rf_stack, feature_map_means, feature_map_stdev,
                                     population_feature_weight_tnsr],
                           outputs=gfm)


In [21]:
compute_fm_grad(vdt, fmt, prfs, fmm, fms, pfwt).shape

(10, 3, 20, 20)

In [22]:
gradient_step_for_one_layer = function(inputs=[voxel_data_tnsr,
                                               population_rf_stack, feature_map_means, feature_map_stdev,
                                               population_feature_weight_tnsr],

                 outputs=[population_prediction_tnsr, population_prediction_error],
                 updates=((feature_map_tnsr, feature_map_tnsr - step_size * gfm), 
                          )

predict = theano.function(inputs=[x], outputs=prediction)

# Train
for i in range(training_steps):
    pred, err = train(D[0], D[1])

SyntaxError: invalid syntax (<ipython-input-22-3bd07fe3bbd1>, line 9)

In [23]:
hoo = tnsr.tensor4('hoo')
ha  = tnsr.tensor4('ha')

hooha = tnsr.batched_tensordot(hoo, ha, axes=[[2,3],[2,3]])

x = np.zeros((D, T, S, S)).astype('float32')
y = np.zeros((D, V, S, S)).astype('float32')

hooha_func = function(inputs = [hoo, ha], outputs = hooha)

In [24]:
print hooha_func(x,y).shape
print (D, T, V)

(3, 10, 101)
(3, 10, 101)


In [25]:
weight_vector = tnsr.vector('wv')


a = tnsr.scalar('a')
b = tnsr.scalar('b')
c = tnsr.scalar('c')

x_a = tnsr.scalar('x_a')
x_b = tnsr.scalar('x_b')
x_c = tnsr.scalar('x_c')

In [26]:
p_expr = [0]
indices = [0,1,2]
for ii, inp in enumerate([x_a,x_b,x_c]):
    p_expr.append(p_expr[-1]+weight_vector[indices[ii]]*inp)

In [27]:
p_expr

[0,
 Elemwise{add,no_inplace}.0,
 Elemwise{add,no_inplace}.0,
 Elemwise{add,no_inplace}.0]

In [29]:

pp(p_expr[-1])

'(((TensorConstant{0} + (Constant{0}[wv] * x_a)) + (Constant{1}[wv] * x_b)) + (Constant{2}[wv] * x_c))'

In [31]:
foo = tnsr.jacobian(p_expr[-1],weight_vector)

In [32]:
baz = function(inputs=[weight_vector,x_a,x_b,x_c], outputs=foo)

In [33]:
in_a,in_b,in_c = np.array(1,dtype='float32'),np.array(2,dtype='float32'),np.array(3,dtype='float32')
wv = np.array([10,20,30],dtype='float32')
baz(wv, in_a,in_b,in_c)

array([ 1.,  2.,  3.], dtype=float32)

In [35]:
layer_expr = []
for inp,wt in zip([a,b,c],[x_a, x_b, x_c]):
    layer_expr.append(inp*wt)
final_expr = tnsr.stack(layer_expr)

In [36]:
pp(final_expr)

'join(TensorConstant{0}, [(a * x_a), (b * x_b), (c * x_c)])'

In [ ]:
blarg = 

In [ ]:
class feature_map(object):
    def __init__(self, shape, name):
        self.shape = shape #(time x depth x resolution x resolution)
        self.name = name
        self.f_map_tnsr = tnsr.tensor4(name)

        
class encoding_model(object):
    def __init__(self, feature_dictionary, activation_function, rf_instance):
        self.f_maps = [feature_map(feature_dictionary[key].shape,key) for key in feature_dictionary.keys()]
        self.receptive_fields = rf_instance
        self.weight_tnsr = tnsr.matrix('weight_matrix')  ##voxels x features (no space). shared?
        self.neural_tnsr = tnsr.tensor4('neural')
        self._build_feature_index()
        self._build_activation_expressions()
        self.prediction = function(inputs=[],outputs=[self.prediction_expr]) ##mean/stdev should be givens
        self.cost = function(inputs=[],outputs=[self.cost_expr])
        
    #=====================================================================================================
    #===this should all go into the model space class...most of it already is...
    #===we just need a more nimble class that builds its own expressions for use by 
    #===by other objects, and that allows the model-space tensor to be built piece-wise.
    def _build_feature_index(self):
        print 'build it yourself'
        ##dictionary of indices into weight matrix
        
        
    ##apply rf and act. func. expressions
    ##this creates a list expressions, one for each feature map
    def _build_activation_expressions(self):
        expr_accumulator = []
        for ii,fm in enumerate(self.f_maps):
            apply_rf = theano_expr
            apply_act_fun = theano_expr
            expr_accumulator.append(divide_out_stdev)  ##need to be careful about mutability here
        self._model_space_expressions = expr_accumulator
    
    ##load in feature dictionary. 
    def _construct_activation_tensor(self, feature_dictionary):
        ##this will apply the rfs and act. funcs to actual data
        ##for loop over time_chunks:
            ##construct rf_stack for specified indices
            ##apply rf stack
        
        ##return activation tensor
    
        
    def _get_feature_map_stats(self, feature_dictionary):
        print 'get them yourself'
        ##this will calculate mean/stdev for each (feature_map , rf) channel
        ##if the stdev is too small, the indices for the (feature_map, rf) channel
        ##are recorded and will be ignored during the gradient step. in this case,
        ##mean=0 and stdev=1. otherwise we just apply mean/stdev normally.
        
    def _build_model_space_expressions(self):
#         apply mean/stdev and the feature weights to get a prediction expression
#         subtract_mean = theano_expr
#         divide_out_stdev = theano_expr
#         sub-select 
        self._model_space_tnrs_expr = theano_expr
        
    def _construct_model_space_tensor(self):
        self._construct_activation_tensor(feature_dictionary)
        
    ##==================================================================================================    
        
    def _build_prediction_expression(self):
          ##builds on the expressions for the model space. applies the weights  
            ##this won't even be used during training. just for validation, etc. after training is done.
#         apply_feature_weights = theano_expr  ##here we need weight matrix indices for current f_map
#         expr_accumulator.append(expr_accumulator[-1]+apply_feature_weights)

#         self.prediction_expr = expr_accumulator[-1]
#         print 'prediction expression built'
#         pp(self.prediction_expr)
        

    
    def _build_cost_expr(self):
        ##uses the same "apply weight" expression for the prediction menu as above
        neural_tnsr = theano_expr
        prediction_menu_tnsr = theano_expr
        self.cost_expr = tnsr.sqr(prediction_menu_tnsr - neural_tnsr).sum() 
        
    
    def _build_grad_expr(self):
        self.grad_expr = tnsr.grad(self.cost_expr, self.weight_tnsr)
        

        
    def _build_training_kernel(self):
        ##returns a function that applies gradient to feature weights then outputs updated feat. weights.
         self.training_kernel = theano_expr
        print 'train your own damn kernel'

    
    def _prepare_feature_maps(self, feature_dictionary, append_to=None):
        ##call this to construct the model-space tensor. 
        ##call this in a loop with append=True if feature dict. doesn't fit into memory,
    
    
    def initialize_training(self, feature_maps, neural_data, initial_weights='zeros', prepared = False, chunking_params):
        ##constructs the model_space tensor. 
        ##if prepared = True, insert a model-space tensor for feature maps
        ##otherwise, call "prepared_feature_maps"
        ##returns indices for splitting up 
    
    def train_me(self):
        for v in voxel_list:
            trn_activity = subset_of_data
            for g in rf_grid_list:
                mst = self.model_space.contruct_mst()
        
        print 'train it yourself'
        
    def test_me(self, val_activity):
        print 'test it yourself'

        
class decoder(object):
    def __init__(self, encoding_model_instance):
        self.encoding_model=encoding_model_instance
        print 'init yourself'
        
    def _build_cost_expr(self):
        self.cost = tnsr.sqr(neural_tnsr - self.prediction_expression).sum() ##or something like this...
    
    def _build_gradient_expr(self):
        self.gradient_expr = jacobian(self.prediction_expr,wrt=[ fm.f_map_tnsr for fm in self.encoding_model.f_maps])
        
    
    def _build_decoding_kernel(self):
        ##
        
        
        
    
        
        
def gradient_descender(data, init_weights, gradient_update_func, pred_func, cost_func):
    
        
        
        
    
        
        